
- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [88]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import encode_tags

In [89]:
# define the path to the directory
directory_path = 'e:/Vocational/Lighthouse Labs/Flex Course/Projects/P02_Midterm_Supervised Learning/data_project_midterm/data'

In [90]:
# List all files in the specified directory
files = [f for f in os.listdir(directory_path) if f.endswith('.json')]
print("Files in directory:")
print(files)

Files in directory:
['AK_Juneau_0.json', 'AK_Juneau_1.json', 'AK_Juneau_2.json', 'AK_Juneau_3.json', 'AK_Juneau_4.json', 'AL_Montgomery_0.json', 'AL_Montgomery_1.json', 'AL_Montgomery_2.json', 'AL_Montgomery_3.json', 'AL_Montgomery_4.json', 'AR_LittleRock_0.json', 'AR_LittleRock_1.json', 'AR_LittleRock_2.json', 'AR_LittleRock_3.json', 'AR_LittleRock_4.json', 'AZ_Phoenix_0.json', 'AZ_Phoenix_1.json', 'AZ_Phoenix_2.json', 'AZ_Phoenix_3.json', 'AZ_Phoenix_4.json', 'CA_Sacramento_0.json', 'CA_Sacramento_1.json', 'CA_Sacramento_2.json', 'CA_Sacramento_3.json', 'CA_Sacramento_4.json', 'CO_Denver_0.json', 'CO_Denver_1.json', 'CO_Denver_2.json', 'CO_Denver_3.json', 'CO_Denver_4.json', 'CT_Hartford_0.json', 'CT_Hartford_1.json', 'CT_Hartford_2.json', 'CT_Hartford_3.json', 'CT_Hartford_4.json', 'DE_Dover_0.json', 'DE_Dover_1.json', 'DE_Dover_2.json', 'DE_Dover_3.json', 'DE_Dover_4.json', 'FL_Tallahassee_0.json', 'FL_Tallahassee_1.json', 'FL_Tallahassee_2.json', 'FL_Tallahassee_3.json', 'FL_Talla

In [91]:
# define the path to the JSON file
file_path = 'e:/Vocational/Lighthouse Labs/Flex Course/Projects/P02_Midterm_Supervised Learning/data_project_midterm/data/AK_Juneau_0.json'

In [92]:
# initialize an empty list to store dataframes
df_list = []

In [93]:
# iterate over each JSON file in the directory
for file_name in files:
    file_path = os.path.join(directory_path, file_name)
    
    # open and read the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    # first level normalization
    df = pd.json_normalize(data['data']['results'])
    
    # Second level normalization
    if 'branding' in df.columns:
        branding = pd.json_normalize(df['branding'].explode())
        branding.columns = [f'branding.{col}' for col in branding.columns]
        df = df.drop('branding', axis=1).join(branding)

    if 'description' in df.columns:
        description = pd.json_normalize(df['description'])
        description.columns = [f'description.{col}' for col in description.columns]
        df = df.drop('description', axis=1).join(description)

    if 'flags' in df.columns:
        flags = pd.json_normalize(df['flags'])
        flags.columns = [f'flags.{col}' for col in flags.columns]
        df = df.drop('flags', axis=1).join(flags)

    if 'lead_attributes' in df.columns:
        lead_attributes = pd.json_normalize(df['lead_attributes'])
        lead_attributes.columns = [f'lead_attributes.{col}' for col in lead_attributes.columns]
        df = df.drop('lead_attributes', axis=1).join(lead_attributes)

    if 'location.address' in df.columns:
        location_address = pd.json_normalize(df['location.address'])
        location_address.columns = [f'location.address.{col}' for col in location_address.columns]
        df = df.drop('location.address', axis=1).join(location_address)

    if 'location.coordinate' in df.columns:
        location_coordinate = pd.json_normalize(df['location.coordinate'])
        location_coordinate.columns = [f'location.coordinate.{col}' for col in location_coordinate.columns]
        df = df.drop('location.coordinate', axis=1).join(location_coordinate)

    if 'location.county' in df.columns:
        location_county = pd.json_normalize(df['location.county'])
        location_county.columns = [f'location.county.{col}' for col in location_county.columns]
        df = df.drop('location.county', axis=1).join(location_county)
    
    # Append the dataframe to the list
    df_list.append(df)

    # Combine all dataframes
df = pd.concat(df_list, ignore_index=True)

In [97]:
df

,last_update_date,tags,permalink,status,list_date,open_houses,list_price,property_id,photos,community,...,primary_photo,source,products,location.address.coordinate,other_listings,branding.name,branding.photo,branding.type,community.advertisers,community.description.name
0,2023-09-19T20:52:50Z,"[carport, community_outdoor_space, cul_de_sac,...",9453-Herbert-Pl_Juneau_AK_99801_M90744-30767,sold,2023-06-29T21:16:25.000000Z,None,554950.0,9074430767,"[{'tags': [{'label': 'house_view', 'probabilit...",None,...,NaN,NaN,NaN,NaN,NaN,EXP Realty LLC - Southeast Alaska,None,Office,NaN,NaN
1,None,None,8477-Thunder-Mountain-Rd_Juneau_AK_99801_M9424...,sold,None,None,NaN,9424983842,None,None,...,NaN,NaN,NaN,NaN,NaN,None,None,Office,NaN,NaN
2,None,None,4515-Glacier-Hwy_Juneau_AK_99801_M94790-68516,sold,None,None,NaN,9479068516,None,None,...,NaN,NaN,NaN,NaN,NaN,None,None,Office,NaN,NaN
3,None,None,17850-Point-Stephens-Rd_Juneau_AK_99801_M98793...,sold,None,None,NaN,9879331943,None,None,...,NaN,NaN,NaN,NaN,NaN,None,None,Office,NaN,NaN
4,None,None,9951-Stephen-Richards-Memorial-Dr_Juneau_AK_99...,sold,None,None,NaN,9521639574,None,None,...,NaN,NaN,NaN,NaN,NaN,None,None,Office,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8189,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
df.columns

Index(['last_update_date', 'tags', 'permalink', 'status', 'list_date',
       'open_houses', 'list_price', 'property_id', 'photos', 'community',
       'virtual_tours', 'listing_id', 'price_reduced_amount', 'matterport',
       'primary_photo.href', 'source.plan_id', 'source.agents',
       'source.spec_id', 'source.type', 'description.year_built',
       'description.baths_3qtr', 'description.sold_date',
       'description.sold_price', 'description.baths_full', 'description.name',
       'description.baths_half', 'description.lot_sqft', 'description.sqft',
       'description.baths', 'description.sub_type', 'description.baths_1qtr',
       'description.garage', 'description.stories', 'description.beds',
       'description.type', 'lead_attributes.show_contact_an_agent',
       'flags.is_new_construction', 'flags.is_for_rent',
       'flags.is_subdivision', 'flags.is_contingent', 'flags.is_price_reduced',
       'flags.is_pending', 'flags.is_foreclosure', 'flags.is_plan',
       'flag

In [95]:
# Define the path to save the CSV file
csv_file_path = 'e:/Vocational/Lighthouse Labs/Flex Course/Projects/P02_Midterm_Supervised Learning/data_project_midterm/data/processed_data.csv'

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

At this point, ensure that you have all sales in a dataframe.
- Is each cell one value, or do some cells have lists?
- Maybe the "tags" will help create some features.
- What are the data types of each column?
- Some sales may not actually include the sale price.  These rows should be dropped.
- Some sales don't include the property type.
- There are a lot of None values.  Should these be dropped or replaced with something?

In [ ]:
# load and concatenate data here
# drop or replace values as necessary

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [ ]:
# OHE categorical variables here
# tags will have to be done manually

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE such as using central tendency?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- If you replace cities or states with numerical values, make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Drop columns that aren't needed.
- Don't keep the list price because it will be too close to the sale price.

In [ ]:
# perform train test split here
# do something with state and city
# drop any other not needed columns

**STRETCH**

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [ ]:
# import, join and preprocess new data here

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [ ]:
# perform EDA here

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized